# Consigna

1. Preparación de los datos:

Vamos a utilizar el dataset MovieLens (https://grouplens.org/datasets/movielens/). Para ello, podemos descargarlo del sitio, seleccionando un conjunto de datos de los disponibles para descargar. Por ejemplo, "ml-latest-small.zip" es una versión reducida del conjunto de datos. (es posible cargar el dataset en memoria, utilizando pandas y el método read_csv()).


2. Implementación del filtrado colaborativo basado en usuario o ítem:

Elegir entre el filtrado colaborativo basado en usuario o en ítem e implementar el algoritmo correspondiente utilizando la biblioteca seleccionada.


3. Evaluación del sistema de recomendación:

Utilizando las métricas RMSE, MAE y precisión, evaluar el modelo creado.


4. Probar el sistema de recomendación:

Generar recomendaciones utilizando el modelo creado para un usuario objetivo.
Es importante que los alumnos describan con celdas de texto lo que van desarrollando y además que realicen un análisis de los resultados obtenidos.

## 1. Preparación de los datos

In [1]:
import pandas as pd

links = pd.read_csv("ml-latest-small/ratings.csv")
movies = pd.read_csv("ml-latest-small/movies.csv")
ratings = pd.read_csv("ml-latest-small/ratings.csv")
tags = pd.read_csv("ml-latest-small/tags.csv")

In [2]:
display(links.head())
display(movies.head())
display(ratings.head())
display(tags.head())

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


## 2. Implementación del filtrado colaborativo basado en usuario o ítem

In [3]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

# Crear el conjunto de datos de Surprise
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Dividir los datos en conjuntos de entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.30)

# Definir el algoritmo de filtrado colaborativo basado en ítem
algo = KNNBasic(sim_options={'user_based': False})

# Entrenar el modelo
algo.fit(trainset)

# Realizar predicciones
predictions = algo.test(testset)

Computing the msd similarity matrix...
Done computing similarity matrix.


## 3. Evaluación del sistema de recomendación

In [4]:
# Calcular el error RMSE (Root Mean Squared Error) y MAE (Mean Absolute Error)
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)
display(rmse)
display(mae)

RMSE: 0.9168
MAE:  0.7048


0.9167929913016661

0.704771072803011

In [5]:
# Fuente: https://github.com/NicolasHug/Surprise/blob/master/examples/precision_recall_at_k.py
from collections import defaultdict

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=3.5)
print(f"Precision at k: {1}", sum(prec for prec in precisions.values()) / len(precisions))
print(f"Recalls at k: {1}", sum(rec for rec in recalls.values()) / len(recalls))

Precision at k: 1 0.6822755659640909
Recalls at k: 1 0.4217620691482492


## 4. Probar el sistema de recomendación

In [6]:
# Seleccionar un usuario objetivo
user_id = 1

# Obtener todos los ids de películas
all_movie_ids = ratings['movieId'].unique()

# Predecir calificaciones para todas las películas no vistas por el usuario
user_ratings = ratings[ratings['userId'] == user_id]
user_watched_movie_ids = user_ratings['movieId'].unique().tolist()
user_unwatched_movie_ids = [movie_id for movie_id in all_movie_ids if movie_id not in user_watched_movie_ids]

# Generar predicciones
predictions = [algo.predict(user_id, movie_id) for movie_id in user_unwatched_movie_ids]

# Obtener las mejores predicciones
top_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]

# Mostrar las recomendaciones
top_movie_ids = [pred.iid for pred in top_predictions]
recommended_movies = movies[movies['movieId'].isin(top_movie_ids)]
display(recommended_movies)

,movieId,title,genres
2548,3410,Soft Fruit (1999),Comedy|Drama
3094,4154,Recess: School's Out (2001),Animation|Children
6437,51705,Priceless (Hors de prix) (2006),Comedy|Romance
7183,72330,"Private Lives of Pippa Lee, The (2009)",Drama
7331,77841,St Trinian's 2: The Legend of Fritton's Gold (...,Adventure|Comedy
7997,97024,Rust and Bone (De rouille et d'os) (2012),Drama|Romance
8265,105211,Enough Said (2013),Comedy|Drama|Romance
8485,113275,The Hundred-Foot Journey (2014),Comedy|Drama
8787,129428,The Second Best Exotic Marigold Hotel (2015),Comedy|Drama
8970,137595,Magic Mike XXL (2015),Comedy|Drama
